# Testing and Grading

Testing is an important part of the software development process.
It's usually not very fun, so it often goes overlooked and neglected.
But it is critical and can also server as a vital part of debugging.
Note that debugging (finding the cause of software issues) is not the same as testing (creating infrastructure to help discover and prevent software issues),
but they have a lot of overlap.
So you can also think of testing as investing in infrastructure to help future debugging.

In a similar vein of testing code, we also have grading code.
They are pretty similar, but grading is usually a bit harder because you have to assign partial credit
and give feedback that is specific enough to be helpful but not too specific as to give away the answer.
Grading is not within the scope of this course, but understanding the basics of grading (and how they are similar to testing)
can help you better understand how your own assignments are graded and hopefully help you get better scores.

In [ ]:
# Import what we will need for this notebook.
import base64

import autograder.question
import autograder.assignment

## Testing in Python

There are many ways to test code, and many different types of tests (unit, integration, behavior, etc).
In this exercise (and course) we are going to focus on [unit testing](https://en.wikipedia.org/wiki/Unit_testing),
which is writing tests for individual code units.
A "unit" can be many different things (function, class, module, etc), but in our case it will usually be a single function.
So we will want to make test functions that test functions.

Note that using print statements generally falls under debugging rather than testing
(but is very much encouraged for debugging).

Here are a few popular testing frameworks in Python, note that this list is far from exhaustive:
 - [unittest](https://docs.python.org/3/library/unittest.html) (Python standard library)
 - [pytest](https://en.wikipedia.org/wiki/Unit_testing)
 - [Robot Framework](https://robotframework.org/)
 - [nose2](https://docs.nose2.io/en/latest/)
 - [Testify](https://github.com/Yelp/Testify)

Each framework has its own style, usage, and specific vocabulary, but it all comes down to the same thing:
running code with known inputs and making sure the output matches.
In this course, we are generally not going to make you write unit tests (we would if we had the time/units),
but you will probably find yourself in situations where writing a test is the best way to fix/finish your code.
So we encourage you to find a framework that you like and get familiar with it.

This class uses an autograding framework that is used to write your local tests (that you can run) and hidden tests (that are used to give you a score).
Below, we will go through an example using the course's grading framework.

## Example Assignment: Fibonacci Sequence

Understanding this example will give you insight into how we build our graders that grade your actual assignments.

Let's say that we are given the task of making an assignment where students need to make a function
to compute the ith [Fibonacci](https://en.wikipedia.org/wiki/Fibonacci_sequence) number.
So `fibonacci(1) == 1`, `fibonacci(4) == 3`, etc.

In [ ]:
# The un-implemented function we give to the students would look like:
def fibonacci(i):
    return NotImplemented

Let's make an assignment to grade this function.

When grading the `fibonacci()` function, we will need to make sure to test all the use cases:
 - 0 and 1 are special cases that have fixed values.
 - The general case starts at 2, so we should test a couple general cases.
 - Negative values are undefined. Let's assume for this exercise that students are supposed to raise a `ValueError` if negative inputs are given.

So far, this is pretty much the same as if you were just testing your own Fibonacci function.
The main difference is that we will now need to give partial credit and feedback when use cases are missed
(instead of just putting up an error like in testing).

In [ ]:
class TestAssignment(autograder.assignment.Assignment):
    def __init__(self, **kwargs):
        # Set prep_submission to false since we don't have any files
        # that go along with this assignment (just this notebook).
        super().__init__(prep_submission = False, questions = [
            TestFibonacci(10),
        ], **kwargs)

class TestFibonacci(autograder.question.Question):
    def score_question(self, _):
        # First we will usually test to see if the function is even implemented.
        # If it is not implemented, then just finish scoring right away.
        value = fibonacci(0)
        if (self.check_not_implemented(value)):
            return

        # Start off with full credit and deduct points for mistakes.
        self.full_credit()
            
        # Test the special starting cases.
        # Give feedback that is helpful, but not too specific!
        
        if (fibonacci(0) != 0):
            self.add_message("Missed an initial condition of Fibonacci.", -2)

        if (fibonacci(1) != 1):
            self.add_message("Missed an initial condition of Fibonacci.", -2)

        # Test some general cases.

        # 2 is a good case to test since it is the first general case.
        if (fibonacci(2) != 1):
            self.add_message("Missed a general case of Fibonacci.", -2)
            
        if (fibonacci(10) != 55):
            self.add_message("Missed a general case of Fibonacci.", -2)
            
        # Test the special negative case.

        try:
            fibonacci(-1)

            # This will only be reached if the above call does not throw an exception.
            self.add_message("Missed a negative case of Fibonacci.", -2)
        except ValueError:
            # Expected
            pass

Now let's try our grader first on the unimplemented function:

In [ ]:
assignment = TestAssignment(name = "Fibonacci Grader")
result = assignment.grade()
print(result.report())

As expected, we get a zero for no implementation.
This means our grader is working!
Now let's try with an actual Fibonacci implementation.

In [ ]:
def fibonacci(i):
    if (i < 0):
        raise ValueError("Fibonacci input must be non-negative, got: '%s'." % (i))

    if (i == 0):
        return 0
    elif (i == 1):
        return 1

    return fibonacci(i - 1) + fibonacci(i - 2)

In [ ]:
assignment = TestAssignment(name = "Fibonacci Grader")
result = assignment.grade()
print(result.report())

Try playing around on your own with this `fibonacci()` implementation.
See how the grader responds when some of the edge cases (or even the general case) is wrong or missing.
Remember, these are the same type of messages you will see in your own assignments.

## Puzzle Tests

Debugging can be one of the most frustrating experiences you can ever have.
But on the other side, you can also think of it as a fun puzzle to solve (especially when there is no money or grades on the line).
Below are some obfuscated functions for you to write tests cases to try and figure out what they do.
Each function takes a single value as input.
They will need the following function to run.
If you really wanted to, you can decode the running function and figure out how the functions are encoded
(it's just Python, so there is only so much you can do to obfuscate code (and we only tried medium-hard to obfuscate)).

Have fun!
(And don't worry, this is not graded.)

In [ ]:
def run_obfuscated_function(value, *args, **kwargs):
    value = bytes.fromhex(str(base64.b64decode(value), 'iso8859_6'))
    value = str(bytes([element ^ 0xFF for element in value]), 'iso8859_6')

    global_values = {}
    exec(value, global_values)

    return global_values[list(sorted(global_values.keys()))[-1]](*args, **kwargs)

In [ ]:
def test_puzzle_1(value):
    return run_obfuscated_function('OWI5YTk5ZGY4ZDlhODk5YThkOGM5YWEwOTM5NjhjOGJkNzk2OGI5YTkyOGNkNmM1ZjVkZmRmZGZkZjkwOGE4YjhmOGE4YmRmYzJkZmE0YTJmNWRmZGZkZmRmZjVkZmRmZGZkZjk5OTA4ZGRmOTY4YjlhOTJkZjk2OTFkZjk2OGI5YTkyOGNjNWY1ZGZkZmRmZGZkZmRmZGZkZjkwOGE4YjhmOGE4YmQxOTY5MThjOWE4ZDhiZDdjZmQzZGY5NjhiOWE5MmQ2ZjVmNWRmZGZkZmRmOGQ5YThiOGE4ZDkxZGY5MDhhOGI4ZjhhOGJmNQ==', value)

def test_puzzle_2(value):
    return run_obfuscated_function('OWI5YTk5ZGY5NjhjYTA5MzlhOWU4ZmEwODY5YTllOGRkNzg2OWE5ZThkZDZjNWY1ZGZkZmRmZGY5Njk5ZGZkNzg2OWE5ZThkZGZkYWRmY2JjZmNmZGZjMmMyZGZjZmQ2YzVmNWRmZGZkZmRmZGZkZmRmZGY4ZDlhOGI4YThkOTFkZmFiOGQ4YTlhZjVmNWRmZGZkZmRmOTY5OWRmZDc4NjlhOWU4ZGRmZGFkZmNlY2ZjZmRmYzJjMmRmY2ZkNmM1ZjVkZmRmZGZkZmRmZGZkZmRmOGQ5YThiOGE4ZDkxZGZiOTllOTM4YzlhZjVmNWRmZGZkZmRmOGQ5YThiOGE4ZDkxZGZkNzg2OWE5ZThkZGZkYWRmY2JkZmMyYzJkZmNmZDZmNQ==', value)